In [1]:
pip install notebook pandas scikit-learn matplotlib seaborn

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [114]:
import pandas as pd

train_data=pd.read_csv("train_LZdllcl.csv")
test_data=pd.read_csv("test_2umaH9m.csv")
sample_submission=pd.read_csv("sample_submission_M0L0uXE.csv")

print(train_data.head())
print(test_data.head())

   employee_id         department     region         education gender  \
0        65438  Sales & Marketing   region_7  Master's & above      f   
1        65141         Operations  region_22        Bachelor's      m   
2         7513  Sales & Marketing  region_19        Bachelor's      m   
3         2542  Sales & Marketing  region_23        Bachelor's      m   
4        48945         Technology  region_26        Bachelor's      m   

  recruitment_channel  no_of_trainings  age  previous_year_rating  \
0            sourcing                1   35                   5.0   
1               other                1   30                   5.0   
2            sourcing                1   34                   3.0   
3               other                2   39                   1.0   
4               other                1   45                   3.0   

   length_of_service  KPIs_met >80%  awards_won?  avg_training_score  \
0                  8              1            0                  49   
1 

In [4]:
#3.Check the Missing Value 

In [116]:
print(train_data.isnull().sum())
print(test_data.isnull().sum())

employee_id                0
department                 0
region                     0
education               2409
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
is_promoted                0
dtype: int64
employee_id                0
department                 0
region                     0
education               1034
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    1812
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
dtype: int64


In [120]:
print(train_data.dtypes)
print(train_data.nunique())

employee_id               int64
department               object
region                   object
education                object
gender                   object
recruitment_channel      object
no_of_trainings           int64
age                       int64
previous_year_rating    float64
length_of_service         int64
KPIs_met >80%             int64
awards_won?               int64
avg_training_score        int64
is_promoted               int64
dtype: object
employee_id             54808
department                  9
region                     34
education                   3
gender                      2
recruitment_channel         3
no_of_trainings            10
age                        41
previous_year_rating        5
length_of_service          35
KPIs_met >80%               2
awards_won?                 2
avg_training_score         61
is_promoted                 2
dtype: int64


In [118]:
print("Columns in train_data before one-hot encoding:")
print(train_data.columns)

print("\nColumns in test_data before one-hot encoding:")
print(test_data.columns)

Columns in train_data before one-hot encoding:
Index(['employee_id', 'department', 'region', 'education', 'gender',
       'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating',
       'length_of_service', 'KPIs_met >80%', 'awards_won?',
       'avg_training_score', 'is_promoted'],
      dtype='object')

Columns in test_data before one-hot encoding:
Index(['employee_id', 'department', 'region', 'education', 'gender',
       'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating',
       'length_of_service', 'KPIs_met >80%', 'awards_won?',
       'avg_training_score'],
      dtype='object')


In [12]:
# Build Your Model (Modeling)

In [122]:
# For training data:
# X_train contains all columns except 'is_promoted'
X_train = train_data.drop('is_promoted', axis=1)

y_train = train_data['is_promoted']

# For test data:
# X_test contains all features you'll use for prediction.
# We drop 'employee_id' as it's just an identifier, not a feature for the model.
X_test = test_data.drop('employee_id', axis=1)

# --- Crucial step to ensure columns match after one-hot encoding ---
# Find columns present in X_train but not in X_test
missing_cols_in_test = set(X_train.columns) - set(X_test.columns)
for c in missing_cols_in_test:
    X_test[c] = 0 # Add these missing columns to X_test and fill with zeros

# Find columns present in X_test but not in X_train
missing_cols_in_train = set(X_test.columns) - set(X_train.columns)
for c in missing_cols_in_train:
    X_train[c] = 0 # Add these missing columns to X_train and fill with zeros

# Ensure the order of columns is identical in both X_test and X_train
X_test = X_test[X_train.columns]

In [124]:
print("--- Original Data Types in Train Data ---")
print(train_data.dtypes) # This will show all column types

# Identify categorical (object) columns from the training data
categorical_cols = train_data.select_dtypes(include=['object']).columns

print(f"\nIdentified Categorical Columns for Encoding: {list(categorical_cols)}")

--- Original Data Types in Train Data ---
employee_id               int64
department               object
region                   object
education                object
gender                   object
recruitment_channel      object
no_of_trainings           int64
age                       int64
previous_year_rating    float64
length_of_service         int64
KPIs_met >80%             int64
awards_won?               int64
avg_training_score        int64
is_promoted               int64
dtype: object

Identified Categorical Columns for Encoding: ['department', 'region', 'education', 'gender', 'recruitment_channel']


In [126]:
# It's good practice to calculate the mean from the training data and apply to both.
if 'previous_year_rating' in train_data.columns and train_data['previous_year_rating'].isnull().any():
    mean_rating = train_data['previous_year_rating'].mean()
    train_data['previous_year_rating'].fillna(mean_rating, inplace=True)
    test_data['previous_year_rating'].fillna(mean_rating, inplace=True) # Use train's mean for test data

# For categorical columns: fill missing values with the most frequent value (mode)
for col in categorical_cols:
    if train_data[col].isnull().any():
        mode_val = train_data[col].mode()[0]
        train_data[col].fillna(mode_val, inplace=True)
        test_data[col].fillna(mode_val, inplace=True)

C:\Users\arjun\AppData\Local\Temp\ipykernel_31128\1788480373.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data['previous_year_rating'].fillna(mean_rating, inplace=True)
C:\Users\arjun\AppData\Local\Temp\ipykernel_31128\1788480373.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves

In [41]:
#Convert ALL Categorical Columns to Numbers (One-Hot Encoding)

In [128]:
# Apply one-hot encoding to all identified categorical columns
train_data = pd.get_dummies(train_data, columns=categorical_cols, drop_first=True)
test_data = pd.get_dummies(test_data, columns=categorical_cols, drop_first=True)

#After get_dummies, check if any 'object' columns remain just in case
print("\n--- Data Types in Train Data AFTER One-Hot Encoding ---")
print(train_data.select_dtypes(include=['object']).columns)

print("\n--- Data Types in Test Data AFTER One-Hot Encoding ---")
print(test_data.select_dtypes(include=['object']).columns)


--- Data Types in Train Data AFTER One-Hot Encoding ---
Index([], dtype='object')

--- Data Types in Test Data AFTER One-Hot Encoding ---
Index([], dtype='object')


In [130]:
print("\n--- Building Your Model ---")

# Separate Features (X) and Target (y) for training data
X_train = train_data.drop('is_promoted', axis=1) # All columns except 'is_promoted'
y_train = train_data['is_promoted'] # Only the 'is_promoted' column

# Features for test data (exclude employee_id as it's an identifier, not a feature)
X_test = test_data.drop('employee_id', axis=1)

# This handles cases where get_dummies might create different columns in train/test
missing_cols_in_test = set(X_train.columns) - set(X_test.columns)
for c in missing_cols_in_test:
    X_test[c] = 0 # Add missing columns to X_test and fill with zeros

missing_cols_in_train = set(X_test.columns) - set(X_train.columns)
for c in missing_cols_in_train:
    X_train[c] = 0 # Add missing columns to X_train and fill with zeros

X_test = X_test[X_train.columns] # Ensure the order of columns is the same as X_train

# Choose and Train a Simple Model (Logistic Regression)
model = LogisticRegression(max_iter=5000, random_state=42) # random_state for reproducibility
print("Training Logistic Regression model...")
model.fit(X_train, y_train)
print("Model training complete.")


--- Building Your Model ---
Training Logistic Regression model...
Model training complete.


C:\Users\arjun\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [75]:
#Make Predictions

In [131]:
print("\n---: Making Predictions ---")
predictions = model.predict(X_test)
print("Predictions made for test data.")


---: Making Predictions ---
Predictions made for test data.


In [80]:
#Create Submission File 

In [136]:
print("\n--- Creating Submission File ---")


sample_submission['is_promoted'] = predictions  # Replace 0s with model predictions

submission_filename = 'my_hr_analytics_submission.csv'
sample_submission.to_csv(submission_filename, index=False)  # index=False is important

print(f" Submission file '{submission_filename}' created successfully!")


--- Creating Submission File ---
 Submission file 'my_hr_analytics_submission.csv' created successfully!
